# IMPORTANT: FIX file with departure edits!

In [1]:
import pandas as pd

In [2]:
thesis_path = r"C:\Users\cvaka\OneDrive\Master\Thesis"

In [3]:
from sqlalchemy import create_engine

POSTGRES_ADDRESS = 'localhost'
# POSTGRES_PORT = '5439'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = 'jonp8UMs8qDV4jEcwOC0'
POSTGRES_DBNAME = 'thesis'

postgres_str = ('postgresql://{username}:{password}@{ipaddress}/{dbname}' # 'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str) 

In [4]:
df_dep = pd.read_sql_query("""
SELECT * FROM departures
;
""", cnx)

# Astra Derived Exploration

In [5]:
pd.read_sql_query("""
SELECT 
    (SELECT COUNT(DISTINCT f_id_sep) FROM astra_derived WHERE type='arr' AND t_taxi IS NOT NULL) AS arr,
    (SELECT COUNT(DISTINCT f_id_sep) FROM astra_derived WHERE type='dep' AND t_taxi IS NOT NULL) AS dep,
    (SELECT COUNT(DISTINCT f_id_sep) FROM astra_derived WHERE type='double') AS both
    ;
""", cnx)

,arr,dep,both
0,493237,469108,6016


In [6]:
pd.read_sql_query("""
SELECT DISTINCT f_id_sep FROM astra_derived WHERE type = 'double';
""", cnx)

,f_id_sep
0,2RBTS_1566720897
1,4XCPX_1531146916
2,4XCPX_1542032757
3,A1_1516098575
4,A1_1520245466
...,...
6011,ZXP26_1570002122
6012,ZXP26_1570188649
6013,ZXP26_1571159249
6014,ZXP26_1574074577


# Departures Exploration

In [7]:
departures_CTEs = """
WITH 
flt_processed_sel AS ( -- Allows for simple change of source table and or number of rows analyzed.
    SELECT *
    FROM flt_processed
), 
dep_t0 AS (
    SELECT flt.flt_sep_id, tstzrange(flt.flt_sep_first, flt.flt_sep_last) AS flt_range, flt_max.trwy AS final_trwy, max(flt.asrt) AS t0_predict
    FROM 
        flt_processed_sel AS flt
        LEFT JOIN
        flt_processed_sel AS flt_max
    ON flt.flt_sep_last = flt_max.t AND flt.flt_sep_id = flt_max.flt_sep_id
    WHERE (flt.flown) AND (flt.landplane) AND (NOT flt.local) AND (NOT flt.civ_reg_gnr) AND (flt.dep_arr=0)
    -- t0 is not null cannot be done within the where clause, done in next query.
    GROUP BY flt.flt_sep_id, flt_range, flt_max.trwy
),
dep_t0_ast AS (
    SELECT * 
    FROM dep_t0 AS dep
    LEFT JOIN
    astra_derived AS astra
    ON 
        (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND 
        (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
    WHERE
        (dep.t0_predict IS NOT NULL) AND 
        (astra.type = 'dep') AND
        (dep.final_trwy = ANY(STRING_TO_ARRAY(astra.rwy, '/'::text)))       
),
dep_t0_ast_t AS (
    SELECT *, dep.t0_predict - make_interval(mins := h.horizon) AS t_predict 
    FROM dep_t0_ast AS dep
    CROSS JOIN (SELECT unnest('{0, 30, 120, 180}'::int[]) AS horizon) AS h 
), 
dep_t0_ast_t_flt AS (
    SELECT dep.*, flt.trwy, flt.actype, flt.obt_predict, flt.depgnr, flt.sid
    FROM dep_t0_ast_t AS dep
    LEFT JOIN
    flt_processed AS flt
    ON (dep.flt_sep_id = flt.flt_sep_id) AND (dep.t_predict <@ flt.valid_range)
    WHERE flt.obt_predict IS NOT NULL
), 
flt_event_table AS (
    SELECT valid_range,
        CASE 
            WHEN dep_arr = 0 THEN obt_predict
            WHEN dep_arr = 1 THEN at_predict
        END AS t_event,
        civ_reg_gnr AS civil,
        (NOT civ_reg_gnr) AND (dep_arr = 0) AS com_dep,
        (NOT civ_reg_gnr) AND (dep_arr = 1) AS com_arr
    FROM flt_processed
    WHERE NOT (obt_predict IS NULL AND at_predict IS NULL)
),
dep_c AS (
    SELECT dep.flt_sep_id, dep.t_predict, sum(civil::int) AS n_civil, sum(com_dep::int) AS n_dep, sum(com_arr::int) AS n_arr
    FROM
        dep_t0_ast_t_flt AS dep
        LEFT JOIN
        flt_event_table AS flt_event
        ON (dep.t_predict <@ flt_event.valid_range) AND ((dep.obt_predict - INTERVAL '10 minute')  <= flt_event.t_event) AND ((dep.obt_predict + INTERVAL '10 minute') >= flt_event.t_event)
    GROUP BY dep.flt_sep_id, dep.t_predict
),
dep_t0_ast_t_flt_c AS (
    SELECT dep.*, n_civil, n_dep, n_arr
    FROM 
        dep_t0_ast_t_flt AS dep
        LEFT JOIN
        dep_c AS dep_c
        ON dep.flt_sep_id = dep_c.flt_sep_id AND dep.t_predict = dep_c.t_predict
), 
dep_t0_ast_t_flt_c_wtc AS (
    SELECT dep.*, i.wtc
    FROM 
        dep_t0_ast_t_flt_c AS dep
        LEFT JOIN icao AS i 
        ON dep.actype = i.designator
),
dep_t0_ast_t_flt_c_wtc_wfs AS (
    SELECT DISTINCT ON (dep.flt_sep_id, dep.t_predict) dep.*, wfs.rvr5000_1000, wfs.rvr1500_300, wfs.rvr550_200, wfs.rvr350, wfs.rvrcat, wfs.wind_dir, wfs.wind_dir_std, wfs.wind_spd, wfs.wind_spd_std, wfs.wind_stoten, wfs.temp, wfs.dew, wfs.snow, wfs.snow_heavy, wfs.rain_cool, wfs.cb, wfs.lightning, wfs.rvr5000_2000
    --, concat(wfs.forecast_id, '_', wfs.t) AS wfs_row_id, wfs.f_type -- for debugging to check match 
    FROM
        dep_t0_ast_t_flt_c_wtc AS dep
        LEFT JOIN
        wfs AS wfs
        ON dep.t_predict >= wfs.t_publish AND dep.obt_predict <@ wfs.valid_range
    ORDER BY dep.flt_sep_id ASC, dep.t_predict ASC, wfs.forecast_id DESC
), 
dep_t0_ast_t_flt_c_wtc_wfs_rwy AS (
    SELECT DISTINCT ON (dep.flt_sep_id, dep.t_predict) dep.*, rwy.plr1, rwy.plr2, rwy.ptr1, rwy.ptr2, rwy.alr1, rwy.alr2, rwy.atr1, rwy.atr2
    --, concat(rwy.t_publish, '_', rwy.valid_range) AS rwy_row_id -- for debugging to check match 
    FROM 
        dep_t0_ast_t_flt_c_wtc_wfs AS dep
        LEFT JOIN
        rwy_config AS rwy
        ON dep.t_predict >= rwy.t_publish AND dep.obt_predict <@ rwy.valid_range
    ORDER BY dep.flt_sep_id ASC, dep.t_predict ASC, rwy.t_publish DESC, rwy.wfs_id DESC
)
"""

## Number of distinct flt_sep_id from flt_processed 

In [8]:
pd.read_sql_query(departures_CTEs + """
SELECT
    (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0) AS flt_dep_records,
    (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0 WHERE t0_predict IS NOT NULL) AS with_t0_predict
    ;
""", cnx)

,flt_dep_records,with_t0_predict
0,491198,487890


In [9]:
pd.read_sql_query(departures_CTEs + """
SELECT DISTINCT flt_sep_id FROM dep_t0 WHERE t0_predict IS NULL;
""", cnx)

,flt_sep_id
0,1638_PHTDS_1518591960
1,147_EDC221_1528726766
2,1971_TRA071_1519801468
3,1137_XRO708_1575553544
4,172_KLM95X_1527551402
...,...
3303,1821_XRO690_1575126868
3304,289_VJT492_1531552657
3305,754_NJE7QF_1562048291
3306,1489_TAY915E_1535398720


## Number of departure tracks from astra_derived

In [10]:
pd.read_sql_query("""
SELECT 
    COUNT (*)
FROM astra_derived
WHERE type='dep';
;""", cnx)

,count
0,469108


In [11]:
pd.read_sql_query(departures_CTEs + """
SELECT DISTINCT f_id_sep FROM astra_derived WHERE type='double';
""", cnx)

,f_id_sep
0,2RBTS_1566720897
1,4XCPX_1531146916
2,4XCPX_1542032757
3,A1_1516098575
4,A1_1520245466
...,...
6011,ZXP26_1570002122
6012,ZXP26_1570188649
6013,ZXP26_1571159249
6014,ZXP26_1574074577


### max(asrt) vs flt_max asrt

In [5]:
pd.read_sql_query("""
WITH
flt_processed_sel AS ( -- Allows for simple change of source table and or number of rows analyzed.
    SELECT *
    FROM flt_processed
),
dep_t0 AS (
    SELECT flt.flt_sep_id, tstzrange(flt.flt_sep_first, flt.flt_sep_last) AS flt_range, flt_max.trwy AS final_trwy, max(flt.asrt) AS t0_predict
    FROM 
        flt_processed_sel AS flt
        LEFT JOIN
        flt_processed_sel AS flt_max
    ON flt.flt_sep_last = flt_max.t AND flt.flt_sep_id = flt_max.flt_sep_id
    WHERE (flt.flown) AND (flt.landplane) AND (NOT flt.local) AND (NOT flt.civ_reg_gnr) AND (flt.dep_arr=0)
    GROUP BY flt.flt_sep_id, flt_range, flt_max.trwy
),
dep_t0_flt_max AS (
    SELECT DISTINCT ON (flt_sep_id)
        flt_sep_id, 
        tstzrange(flt_sep_first, flt_sep_last) AS flt_range, 
        trwy AS final_trwy, 
        asrt AS t0_predict
    FROM flt_processed_sel
    WHERE t = flt_sep_last AND flown AND landplane AND (NOT local) AND (NOT civ_reg_gnr) AND dep_arr=0
    ORDER BY flt_sep_id, t DESC
)
SELECT *
FROM
    dep_t0 AS a
    FULL JOIN
    dep_t0_flt_max AS b
    ON a.flt_sep_id = b.flt_sep_id
WHERE 
    ((a.flt_sep_id IS NULL) OR 
    (b.flt_sep_id IS NULL) OR 
    (a.flt_range != b.flt_range) OR 
    (a.final_trwy!=b.final_trwy) OR
    (a.t0_predict!=b.t0_predict)) AND
    (a.t0_predict IS NOT NULL)
;
""", cnx)
    
    

,flt_sep_id,flt_range,final_trwy,t0_predict,flt_sep_id,flt_range,final_trwy,t0_predict
0,102_BOO724_1517984559,"[2018-02-07 03:00:01+00:00, 2018-02-07 06:56:2...",04,NaT,None,None,None,NaT
1,1027_TRA91G_1554280188,"[2019-04-03 04:10:01+00:00, 2019-04-03 08:49:4...",18L,2019-04-03 07:25:42+00:00,None,None,None,2019-04-03 07:25:42+00:00
2,1030_FYG71X_1564641221,"[2019-08-01 03:15:00+00:00, 2019-08-01 06:55:4...",24,NaT,None,None,None,NaT
3,1031_AFR1441_1576958607,"[2019-12-21 16:25:00+00:00, 2019-12-21 20:21:2...",24,2019-12-21 19:28:36+00:00,None,None,None,2019-12-21 19:28:36+00:00
4,1031_N7779V_1551717300,"[2019-03-04 14:57:28+00:00, 2019-03-04 17:20:0...",None,NaT,None,None,None,NaT
...,...,...,...,...,...,...,...,...
3556,982_PHTDS_1550842564,"[2019-02-22 12:27:38+00:00, 2019-02-22 14:14:2...",22,NaT,None,None,None,NaT
3557,986_NJE990T_1561355083,"[2019-06-24 02:30:00+00:00, 2019-06-24 06:13:0...",22,NaT,None,None,None,NaT
3558,986_TRA071_1524045765,"[2018-04-18 09:19:50+00:00, 2018-04-18 10:35:0...",24,NaT,None,None,None,NaT
3559,99_KLM1395_1545999716,"[2018-12-28 09:06:02+00:00, 2018-12-28 13:17:4...",24,2018-12-28 12:06:11+00:00,None,None,None,2018-12-28 12:06:11+00:00


## Number of distinct flt_sep_id from flt_processed with a t0 matched with an astra track (t_taxi IS NOT NULL and type='dep' and flt.final_trwy=astra.rwy)

In [12]:
pd.read_sql_query(departures_CTEs + """
SELECT
    (
        SELECT COUNT(DISTINCT flt_sep_id)
        FROM dep_t0_ast
    ) AS n_dep_t0_ast,
    (
        (SELECT COUNT(*) FROM dep_t0_ast) - (SELECT COUNT( DISTINCT flt_sep_id ) FROM dep_t0_ast)
    ) AS multiple_join_error,
    (
        SELECT COUNT(DISTINCT flt_sep_id) 
        FROM 
            dep_t0 AS dep
            LEFT JOIN
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE astra.t_taxi IS NULL
    ) AS t_taxi_null,
    (
        SELECT COUNT(DISTINCT flt_sep_id) 
        FROM 
            dep_t0 AS dep
            LEFT JOIN
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE (astra.type IS NULL)
    ) AS type_null,
    (
        SELECT COUNT(DISTINCT flt_sep_id) 
        FROM 
            dep_t0 AS dep
            LEFT JOIN
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE (astra.type = 'double')
    ) AS matched_to_double_arr,
    (
        SELECT COUNT(DISTINCT flt_sep_id)
        FROM
            dep_t0 AS dep
            LEFT JOIN
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE (astra.type = 'arr')
    ) AS matched_to_arr,
    (
        SELECT COUNT(DISTINCT flt_sep_id)
        FROM
            dep_t0 AS dep
            LEFT JOIN 
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE 
            (dep.t0_predict IS NOT NULL)        
    ) AS no_t0_predict,
    (
        SELECT COUNT(DISTINCT flt_sep_id)
        FROM
            dep_t0 AS dep
            LEFT JOIN 
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE 
            (dep.t0_predict IS NOT NULL) AND
            (astra.type = 'dep')
    ) AS no_type_dep,
    (
        SELECT COUNT(DISTINCT flt_sep_id)
        FROM
            dep_t0 AS dep
            LEFT JOIN 
            astra_derived AS astra
            ON (Split_part(dep.flt_sep_id, '_', 2) = Split_part(astra.f_id_sep, '_', 1)) AND (TO_TIMESTAMP(Split_part(astra.f_id_sep, '_', 2)::bigint) <@ dep.flt_range)
        WHERE 
            (dep.t0_predict IS NOT NULL) AND
            (type = 'dep') AND
            (NOT dep.final_trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text)))
    ) AS no_final_trwy
;
""", cnx)

,n_dep_t0_ast,multiple_join_error,t_taxi_null,type_null,matched_to_double_arr,matched_to_arr,no_t0_predict,no_type_dep,no_final_trwy
0,439882,8,51729,51335,408,2789,487890,439960,89


When type!= dep then t_taxi IS NULL -> Filter unnecessary (t_taxi_null = type_null + type_double)

### Incorrect Runway

In [13]:
pd.read_sql_query("""
SELECT 
    SUM((NOT final_trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text)))::int) AS failed_final_trwy_match, 
    SUM((NOT trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text)))::int) AS failed_trwy_match 
FROM departures
""", cnx)

,failed_final_trwy_match,failed_trwy_match
0,0,286188


In [14]:
pd.read_sql_query("""
SELECT 
    SUM((NOT final_trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text)))::int) AS failed_final_trwy_match, 
    SUM((NOT trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text)))::int) AS failed_trwy_match 
FROM departures
""", cnx)

,failed_final_trwy_match,failed_trwy_match
0,0,286188


In [15]:
df = pd.read_sql_query("""
SELECT DISTINCT ON (t0_predict, flt_sep_id) flt_sep_id, t0_predict
FROM departures 
WHERE NOT final_trwy = ANY(STRING_TO_ARRAY(rwy, '/'::text))
ORDER BY t0_predict, flt_sep_id, horizon
""", cnx)
df

,flt_sep_id,t0_predict


### Missing any obt

In [16]:
pd.read_sql_query(departures_CTEs + """
SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0_ast_t_flt
""", cnx)

,count
0,439801


## Number of departures with a obt_predict at t_predict for different horizons

In [17]:
df = pd.read_sql_query(departures_CTEs + """
SELECT horizon, count(DISTINCT flt_sep_id)
FROM dep_t0_ast_t_flt
GROUP BY horizon;
"""
, cnx)

print('Total Predictions: ', df['count'].sum())
df

Total Predictions:  1596581


,horizon,count
0,0,439799
1,30,438785
2,120,429489
3,180,288508


In [18]:
df = pd.read_sql_query(departures_CTEs + """
SELECT dep.*, flt.trwy, flt.actype, flt.obt_predict, flt.depgnr, flt.sid
FROM 
    dep_t0_ast_t AS dep
    LEFT JOIN
    flt_processed AS flt
    ON (dep.flt_sep_id = flt.flt_sep_id) AND (dep.t_predict <@ flt.valid_range)
WHERE flt.obt_predict IS NULL AND dep.horizon = 0
""", cnx)

df.to_csv("obt_null_horizon_0.csv")
df
# len(errors['obt_predict_NULL_at_horizon_0'].index)

,flt_sep_id,flt_range,final_trwy,t0_predict,f_id_sep,t_taxi,type,rwy,t_spent,horizon,t_predict,trwy,actype,obt_predict,depgnr,sid
0,20_ADR499_1560363822,"[2019-06-12 15:00:01+00:00, 2019-06-12 18:47:0...",18L,2019-06-12 18:09:01+00:00,ADR499_1560363002,457,dep,18L/36R,"{'09_threshold': 46, '18L_queue': 78, '18L_thr...",0,2019-06-12 18:09:01+00:00,18L,CRJ9,None,A52,REN1E
1,38_AEA64BZ_1561945961,"[2019-07-01 00:28:03+00:00, 2019-07-01 02:15:2...",24,2019-07-01 01:41:56+00:00,AEA64BZ_1561945285,526,dep,06/24,"{'24_queue': 10, '24_threshold_1': 57, 'air': ...",0,2019-07-01 01:41:56+00:00,24,B788,None,None,KDD1S
2,488_AEH181_1524120312,"[2018-04-19 03:00:00+00:00, 2018-04-19 07:11:2...",22,2018-04-19 06:32:36+00:00,AEH181_1524118229,165,dep,04/22,"{'04/22_queue': 81, '04/22_threshold_1': 84, '...",0,2018-04-19 06:32:36+00:00,22,E145,None,None,KDD1G
3,892_AFL2193_1522967016,"[2018-04-05 19:24:58+00:00, 2018-04-05 22:52:1...",24,2018-04-05 22:10:32+00:00,AFL2193_1522966261,691,dep,06/24,"{'24_queue': 42, '24_threshold_1': 67, 'air': ...",0,2018-04-05 22:10:32+00:00,24,A321,None,G04,AND1S
4,361_AFL2193_1531694263,"[2018-07-15 20:30:59+00:00, 2018-07-15 23:06:3...",06,2018-07-15 22:20:30+00:00,AFL2193_1531693261,924,dep,06/24,"{'04/22_threshold_2': 8, '09_queue': 41, '09_t...",0,2018-07-15 22:20:30+00:00,06,A321,None,G03,AND1T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1536_TRA6727_1528208799,"[2018-06-05 10:55:00+00:00, 2018-06-05 14:49:3...",09,2018-06-05 14:10:57+00:00,TRA6727_1528206547,809,dep,09/27,"{'09_queue': 123, '09_threshold': 342, '18L_th...",0,2018-06-05 14:10:57+00:00,09,B738,None,D56,KDD1N
79,1586_TRA76Y_1528209619,"[2018-06-05 11:15:00+00:00, 2018-06-05 15:02:5...",09,2018-06-05 14:28:40+00:00,TRA76Y_1528208962,483,dep,09/27,"{'09_queue': 18, '09_threshold': 171, 'air': 1...",0,2018-06-05 14:28:40+00:00,09,B738,None,D55,REN1N
80,1566_VLG8424_1528209953,"[2018-06-05 11:05:00+00:00, 2018-06-05 15:08:3...",09,2018-06-05 14:32:39+00:00,VLG8424_1528209359,439,dep,09/27,"{'09_queue': 1, '09_threshold': 194, 'air': 14...",0,2018-06-05 14:32:39+00:00,09,A320,None,D92,KDD1N
81,1184_VLG902P_1534571803,"[2018-08-18 03:36:40+00:00, 2018-08-18 06:18:3...",24,2018-08-18 05:43:16+00:00,VLG902P_1534571282,418,dep,06/24,"{'24_queue': 75, '24_threshold_1': 113, 'air':...",0,2018-08-18 05:43:16+00:00,24,A320,None,B35,KDD1S


In [19]:
pd.read_sql_query(departures_CTEs + """
SELECT
    (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0_ast_t),
    (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0_ast_t_flt),
    (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0_ast_t) - (SELECT COUNT(DISTINCT flt_sep_id) FROM dep_t0_ast_t_flt),
    (
        SELECT COUNT(*)
        FROM (
            SELECT dep.flt_sep_id, max(flt.obt_predict) AS max
            FROM 
                dep_t0_ast_t AS dep
                LEFT JOIN
                flt_processed AS flt
            ON (dep.flt_sep_id = flt.flt_sep_id) AND (dep.t_predict <@ flt.valid_range)
            GROUP BY dep.flt_sep_id
        ) AS foo
        WHERE foo.max IS NULL

    )
""", cnx)

,count,count,?column?,count
0,439882,439801,81,81


In [20]:
pd.read_sql_query("""
SELECT COUNT(*) FROM departures;
"""
, cnx)

,count
0,1596581


In [21]:
pd.read_sql_query("""
SELECT COUNT(DISTINCT CONCAT(flt_sep_id, '_', t_predict)) FROM departures;
"""
, cnx)

,count
0,1596581


## WFS

In [22]:
pd.read_sql_query("""
SELECT COUNT( DISTINCT CONCAT(flt_sep_id, '_', t_predict))
FROM departures 
WHERE
    rvr5000_1000 IS NULL AND 
    rvr1500_300 IS NULL AND 
    rvr550_200 IS NULL AND 
    rvr350 IS NULL AND 
    rvrcat IS NULL AND 
    wind_dir IS NULL AND 
    wind_dir_std IS NULL AND 
    wind_spd IS NULL AND 
    wind_spd_std IS NULL AND 
    wind_stoten IS NULL AND 
    temp IS NULL AND 
    dew IS NULL AND 
    snow IS NULL AND 
    snow_heavy IS NULL AND 
    rain_cool IS NULL AND 
    cb IS NULL AND 
    lightning IS NULL AND 
    rvr5000_2000 IS NULL
;
""", cnx)

,count
0,6


In [23]:
pd.read_sql_query("""
SELECT 
--  flt_sep_id, 
    t_predict, 
    obt_predict--, 
--  horizon, 
--  t0_predict
FROM departures 
WHERE
    rvr5000_1000 IS NULL AND 
    rvr1500_300 IS NULL AND 
    rvr550_200 IS NULL AND 
    rvr350 IS NULL AND 
    rvrcat IS NULL AND 
    wind_dir IS NULL AND 
    wind_dir_std IS NULL AND 
    wind_spd IS NULL AND 
    wind_spd_std IS NULL AND 
    wind_stoten IS NULL AND 
    temp IS NULL AND 
    dew IS NULL AND 
    snow IS NULL AND 
    snow_heavy IS NULL AND 
    rain_cool IS NULL AND 
    cb IS NULL AND 
    lightning IS NULL AND 
    rvr5000_2000 IS NULL
;
""", cnx)

,t_predict,obt_predict
0,2018-03-05 06:34:00+00:00,2018-03-10 09:00:00+00:00
1,2019-05-09 18:06:10+00:00,2019-05-23 19:40:00+00:00
2,2019-08-12 12:20:52+00:00,2019-08-14 12:17:00+00:00
3,2019-11-04 05:13:53+00:00,2019-11-05 15:30:00+00:00
4,2019-11-04 06:13:53+00:00,2019-11-05 15:30:00+00:00
5,2018-06-05 10:20:31+00:00,2018-06-10 10:20:00+00:00


In [24]:
pd.read_sql_query("""
SELECT * FROM departures WHERE flt_sep_id = '61_YZR7480_1572856750'
;
""", cnx)

,flt_sep_id,flt_range,final_trwy,t0_predict,f_id_sep,t_taxi,type,rwy,t_spent,horizon,...,ptr1,ptr2,alr1,alr2,atr1,atr2,local_doy,local_dow,local_mod,local_week
0,61_YZR7480_1572856750,"[2019-11-04 04:54:00+00:00, 2019-11-04 09:07:2...",18C,2019-11-04 08:13:53+00:00,YZR7480_1572855326,1255,dep,18C/36C,"{'09_queue': 1, '09_threshold': 294, '18C_queu...",180,...,None,None,None,None,None,None,308.0,1.0,373.0,45.0
1,61_YZR7480_1572856750,"[2019-11-04 04:54:00+00:00, 2019-11-04 09:07:2...",18C,2019-11-04 08:13:53+00:00,YZR7480_1572855326,1255,dep,18C/36C,"{'09_queue': 1, '09_threshold': 294, '18C_queu...",120,...,None,None,None,None,None,None,308.0,1.0,433.0,45.0
2,61_YZR7480_1572856750,"[2019-11-04 04:54:00+00:00, 2019-11-04 09:07:2...",18C,2019-11-04 08:13:53+00:00,YZR7480_1572855326,1255,dep,18C/36C,"{'09_queue': 1, '09_threshold': 294, '18C_queu...",30,...,18L,None,None,None,None,None,308.0,1.0,523.0,45.0
3,61_YZR7480_1572856750,"[2019-11-04 04:54:00+00:00, 2019-11-04 09:07:2...",18C,2019-11-04 08:13:53+00:00,YZR7480_1572855326,1255,dep,18C/36C,"{'09_queue': 1, '09_threshold': 294, '18C_queu...",0,...,18L,None,None,None,None,None,308.0,1.0,553.0,45.0


## rwy_config

In [25]:
pd.read_sql_query("""
SELECT COUNT( DISTINCT CONCAT(flt_sep_id, '_', t_predict) )
FROM departures 
WHERE
    plr1 IS NULL AND
    plr2 IS NULL AND
    ptr1 IS NULL AND
    ptr2 IS NULL AND
    alr1 IS NULL AND
    alr2 IS NULL AND
    atr1 IS NULL AND
    atr2 IS NULL
;
""", cnx)

,count
0,92674


In [26]:
pd.read_sql_query("""
SELECT horizon, COUNT( DISTINCT CONCAT(flt_sep_id, '_', t_predict) )
FROM departures 
WHERE
    plr1 IS NULL AND
    plr2 IS NULL AND
    ptr1 IS NULL AND
    ptr2 IS NULL AND
    alr1 IS NULL AND
    alr2 IS NULL AND
    atr1 IS NULL AND
    atr2 IS NULL
GROUP BY horizon
;
""", cnx)

,horizon,count
0,0,2411
1,30,5853
2,120,31667
3,180,52743


Cause: predictions are only published up to eight hours in advance with a minum of twenty minutes before.

In [27]:
df = pd.read_sql_query("""
SELECT *
FROM departures 
WHERE
    plr1 IS NULL AND
    plr2 IS NULL AND
    ptr1 IS NULL AND
    ptr2 IS NULL AND
    alr1 IS NULL AND
    alr2 IS NULL AND
    atr1 IS NULL AND
    atr2 IS NULL
;
""", cnx)
df.to_csv("missing_rwy_config.csv")
df

,flt_sep_id,flt_range,final_trwy,t0_predict,f_id_sep,t_taxi,type,rwy,t_spent,horizon,...,ptr1,ptr2,alr1,alr2,atr1,atr2,local_doy,local_dow,local_mod,local_week
0,0_AFR1893_1572463595,"[2019-10-30 16:20:00+00:00, 2019-10-30 19:49:0...",18L,2019-10-30 19:16:24+00:00,AFR1893_1572462857,468,dep,18L/36R,"{'09_queue': 1, '09_threshold': 25, '18L_thres...",30,...,None,None,None,None,None,None,303.0,3.0,1186.0,44.0
1,0_AFR1893_1572463595,"[2019-10-30 16:20:00+00:00, 2019-10-30 19:49:0...",18L,2019-10-30 19:16:24+00:00,AFR1893_1572462857,468,dep,18L/36R,"{'09_queue': 1, '09_threshold': 25, '18L_thres...",0,...,None,None,None,None,None,None,303.0,3.0,1216.0,44.0
2,0_BAW429_1572167697,"[2019-10-27 06:05:03+00:00, 2019-10-27 09:44:0...",24,2019-10-27 09:01:39+00:00,BAW429_1572166949,617,dep,06/24,"{'24_queue': 165, '24_threshold_1': 171, 'air'...",120,...,None,None,None,None,None,None,300.0,0.0,481.0,43.0
3,0_BEE3DA_1551698263,"[2019-03-04 06:20:01+00:00, 2019-03-04 11:47:2...",24,2019-03-04 11:04:54+00:00,BEE3DA_1551697788,205,dep,06/24,"{'24_queue': 3, '24_threshold_1': 64, 'air': 4...",180,...,None,None,None,None,None,None,63.0,1.0,544.0,10.0
4,0_EZY53FU_1539160809,"[2018-10-10 04:50:02+00:00, 2018-10-10 09:06:2...",24,2018-10-10 08:24:23+00:00,EZY53FU_1539159930,792,dep,06/24,"{'09_queue': 8, '09_threshold': 108, '24_queue...",180,...,None,None,None,None,None,None,283.0,3.0,444.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92669,999_TRA71F_1551973781,"[2019-03-07 12:30:00+00:00, 2019-03-07 16:12:4...",24,2019-03-07 15:41:10+00:00,TRA71F_1551972994,366,dep,06/24,"{'24_threshold_1': 41, 'air': 141, 'alpha_brav...",120,...,None,None,None,None,None,None,66.0,4.0,881.0,10.0
92670,999_TRA71F_1551973781,"[2019-03-07 12:30:00+00:00, 2019-03-07 16:12:4...",24,2019-03-07 15:41:10+00:00,TRA71F_1551972994,366,dep,06/24,"{'24_threshold_1': 41, 'air': 141, 'alpha_brav...",30,...,None,None,None,None,None,None,66.0,4.0,971.0,10.0
92671,999_TRA71F_1551973781,"[2019-03-07 12:30:00+00:00, 2019-03-07 16:12:4...",24,2019-03-07 15:41:10+00:00,TRA71F_1551972994,366,dep,06/24,"{'24_threshold_1': 41, 'air': 141, 'alpha_brav...",0,...,None,None,None,None,None,None,66.0,4.0,1001.0,10.0
92672,999_TRA73M_1528464889,"[2018-06-08 10:25:01+00:00, 2018-06-08 13:57:4...",09,2018-06-08 13:24:54+00:00,TRA73M_1528462965,476,dep,09/27,"{'09_threshold': 192, 'air': 150, 'alpha_bravo...",120,...,None,None,None,None,None,None,159.0,5.0,804.0,23.0


## departure null stats

In [28]:
df = []
horizons = [0,30,120,180]
for horizon in horizons:
    df.append(df_dep[df_dep['horizon']==horizon].isnull().sum(axis=0).rename(horizon))
df = pd.concat(df, axis=1)#, names=horizons)
df ['total'] = df.sum(axis=1).rename('total')
df = df[df['total']!=0].transpose()
df.to_csv('nulls.csv')
df
#df[df==0]#.sum(axis=1)

,depgnr,sid,rvr5000_1000,rvr1500_300,rvr550_200,rvr350,rvrcat,wind_dir,wind_dir_std,wind_spd,...,lightning,rvr5000_2000,plr1,plr2,ptr1,ptr2,alr1,alr2,atr1,atr2
0,53,2608,0,0,0,0,0,0,54,0,...,0,54,2411,315852,2513,171697,370728,403071,370830,414661
30,80,2536,2,2,2,2,2,2,57,2,...,2,57,5853,314581,5967,173089,362552,401822,362666,406599
120,76,2035,2,2,2,2,2,2,3,2,...,2,3,31667,321969,31793,177701,341027,395416,341153,382053
180,41,1300,2,2,2,2,2,2,2,2,...,2,2,52743,228262,52901,133030,233535,268392,233693,257640
total,250,8479,6,6,6,6,6,6,116,6,...,6,116,92674,1180664,93174,655517,1307842,1468701,1308342,1460953


## Double Matches

In [29]:
pd.read_sql_query(departures_CTEs + """
SELECT
    (
        SELECT COUNT(*)
        FROM (
            SELECT flt_sep_id, COUNT(*) AS c 
            FROM dep_t0
            GROUP BY flt_sep_id
        ) AS t0_duplicates
        WHERE c != 1
    ) AS t0_dup,
    (
        SELECT COUNT(*)
        FROM (
            SELECT flt_sep_id, COUNT(*) AS c 
            FROM dep_t0_ast
            GROUP BY flt_sep_id
        ) AS astra_dup
        WHERE c!=1
    ) AS astra_dup,
    (
        SELECT COUNT(*) FROM dep_t0_ast WHERE type!='dep'
    ) AS astra_merge_errors,
    (
        SELECT COUNT(*)
        FROM (
            SELECT dep.flt_sep_id, dep.t_predict, wfs.forecast_id, COUNT(*) AS c
            FROM 
                dep_t0_ast_t_flt AS dep
                LEFT JOIN
                wfs AS wfs
                ON dep.t_predict >= wfs.t_publish AND dep.obt_predict <@ wfs.valid_range
            GROUP BY dep.flt_sep_id, dep.t_predict, wfs.forecast_id
        ) AS wfs_dup_errors
        WHERE c!=1
    ) AS wfs_dup_errors,
    (
        SELECT COUNT(*)
        FROM (
            SELECT dep.flt_sep_id, dep.t_predict, rwy.t_publish, COUNT(*) AS c
            FROM dep_t0_ast_t_flt AS dep
            LEFT JOIN
            rwy_config AS rwy
            ON dep.t_predict >= rwy.t_publish AND dep.obt_predict <@ rwy.valid_range
            GROUP BY dep.flt_sep_id, dep.t_predict, rwy.t_publish
        ) AS rwy_dup_errors
        WHERE c!= 1
    ) AS rwy_dup_errors
""", cnx)

,t0_dup,astra_dup,astra_merge_errors,wfs_dup_errors,rwy_dup_errors
0,0,8,0,518,287


In [30]:
df = pd.read_sql_query(departures_CTEs + """
        SELECT *
        FROM (
            SELECT dep.flt_sep_id, dep.t_predict, rwy.t_publish, COUNT(*) AS c
            FROM dep_t0_ast_t_flt AS dep
            LEFT JOIN
            rwy_config AS rwy
            ON dep.t_predict >= rwy.t_publish AND dep.obt_predict <@ rwy.valid_range
            GROUP BY dep.flt_sep_id, dep.t_predict, rwy.t_publish
        ) AS rwy_dup_errors
        WHERE c!= 1
""", cnx)
df.to_csv("cfs_duplicate_matches.csv")